## 불용어 제거, 토큰화, POS 태깅 단계는 필수
<p> 영문 한글로 번역 필요

In [91]:
from konlpy.tag import Kkma

sentence= "나는, 지금 기쁘다."
kkma = Kkma()
token = kkma.morphs(sentence)
print("token : ",token)

token :  ['나', '는', ',', '지금', '기쁘', '다', '.']


In [92]:
tagging_kkma = []
for i in token :
    tag_word = kkma.pos(i)
    tagging_kkma.append(tag_word)
    
print("tagging : ",tagging_kkma)

tagging :  [[('나', 'NNG')], [('늘', 'VA'), ('ㄴ', 'ETD')], [(',', 'SP')], [('지금', 'NNG')], [('기쁘', 'VA')], [('다', 'NNG')], [('.', 'SF')]]


In [93]:
stopword = []
tagging_cut1_kkma = []

for i in range(len(tagging_kkma)) :
    for word, tag in tagging_kkma[i] :
        if tag in 'SF' or tag in 'SP'or tag in 'SS' or tag in 'SE' or tag in 'SO' or tag in 'SW' or tag in 'UN' or tag in 'OH' or tag in 'ON':
            stopword.append(word)
        else :
            value = (word,tag)
            tagging_cut1_kkma.append(value)
            
print(stopword)
tagging_cut1_kkma

[',', '.']


[('나', 'NNG'),
 ('늘', 'VA'),
 ('ㄴ', 'ETD'),
 ('지금', 'NNG'),
 ('기쁘', 'VA'),
 ('다', 'NNG')]

# 감성분석 회귀분석 예측

<p> 0 : 기쁨 행복</p>
<p> 1 : 분노 불안 당황 놀람</p>
<p> 2: 슬픔 상처 </p>

In [94]:
from sklearn.model_selection import train_test_split
import pandas as pd
from afinn import Afinn
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

dataset = pd.read_csv("sentiment_analysis.csv",encoding = 'utf-8')
dataset.head()

,number,value,age,sex,keyword,disease,sentiment_big,sentiment_small,person1,system1,person2,system2,person3,system3
0,24070,1,청년,여성,"진로, 취업, 직장",해당없음,기쁨,신이 난,지금 난 기분이 너무 좋아.,굉장히 즐거우신 것 같은데요? 좋은 일이 있나요?,오늘부터 연휴가 시작됐거든. 내일 회사 안 간다.,그거 정말 기분 좋을 만하네요. 무엇을 할 예정인가요?,그냥 집에서 뒹굴뒹굴하고 넷플릭스 보고 놀 거야.,집에서 노는 것이 최고죠. 생각만 해도 행복하겠어요.
1,27879,1,청년,여성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,나도 조카가 생겨! 너무 기뻐.,정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요?,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.,언니에게 꽃을 선물하려고 하시는군요.,NaN,NaN
2,51471,1,중년,여성,"직장, 업무 스트레스",해당없음,불안,불안,거래처와의 다음 계약이 무산될까봐 불안해.,많이 걱정되시겠어요. 계약이 무산될 것 같다고 생각하는 이유가 있으신가요?,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.,계약을 성공적으로 마무리하실 수 있기를 기원해요.,NaN,NaN
3,7140,1,노년,남성,대인관계,만성질환 유,슬픔,염세적인,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...,몸이 아픈데 아내가 이혼하자고 하니 혼자 있는 것 같은 기분을 느끼셨군요.,젊은 시절에 가정에 소홀했지. 그래도 그렇지 이혼을 하자니 너무 허망해.,지금의 기분에서 벗어나기 위해 무엇을 하면 좋을까요?,아내에게 졸혼은 어떤지 대화를 나누어 봐야겠어. 졸혼을 한다면 가끔씩이라도 도움을 ...,아내와의 대화로 기분이 나아졌으면 좋겠어요.
4,39663,1,노년,남성,재정,만성질환 유,슬픔,마비된,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,나이들어 재정적으로 어려워서 걱정되시는군요.,정말이지 막막해.,막막한 현실이지만 분명 해결책을 찾을 수 있을거예요.,NaN,NaN


In [139]:
sentimential_list = {'기쁨': 0, '분노' : 1, '불안' : 1, '당황' : 1, '슬픔' : 2, '상처' : 2, '행복' : 0}

data1 = pd.DataFrame()
data1['label'] = dataset['sentiment_big'].map(sentimential_list)
data1['text'] = dataset['person1']
data1.reset_index()
data1.head()

,label,text
0,0,지금 난 기분이 너무 좋아.
1,0,나도 조카가 생겨! 너무 기뻐.
2,1,거래처와의 다음 계약이 무산될까봐 불안해.
3,2,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...
4,2,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.


In [140]:
data2 = pd.DataFrame()
data2['label'] = dataset['sentiment_big'].map(sentimential_list)
data2['text'] = dataset['system1']
data2.reset_index()
data2.head()

,label,text
0,0,굉장히 즐거우신 것 같은데요? 좋은 일이 있나요?
1,0,정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요?
2,1,많이 걱정되시겠어요. 계약이 무산될 것 같다고 생각하는 이유가 있으신가요?
3,2,몸이 아픈데 아내가 이혼하자고 하니 혼자 있는 것 같은 기분을 느끼셨군요.
4,2,나이들어 재정적으로 어려워서 걱정되시는군요.


In [141]:
data3 = pd.DataFrame()
data3['label'] = dataset['sentiment_big'].map(sentimential_list)
data3['text'] = dataset['person2']
data3.reset_index()
data3.head()

,label,text
0,0,오늘부터 연휴가 시작됐거든. 내일 회사 안 간다.
1,0,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.
2,1,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.
3,2,젊은 시절에 가정에 소홀했지. 그래도 그렇지 이혼을 하자니 너무 허망해.
4,2,정말이지 막막해.


In [142]:
# 데이터 분포
# data1 data2 data3 모두 동일
data1['label'].value_counts()

1    36194
2    26213
0    12449
Name: label, dtype: int64

In [143]:
import re

data1 = data1.fillna(" ")
data1['text'] = data1['text'].apply(lambda x:re.sub(r"\d+"," ",x))

x = data1['text']
y = data1['label']

x_train, x_test, y_train, y_test = train_test_split(x,y, stratify = y, test_size=0.2)

from konlpy.tag import Okt

okt = Okt()
def tw_tokenizer(text) :
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [144]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer= tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(x_train)
tfidf_matrix_train = tfidf_vect.transform(x_train)

C:\Users\sapb2\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [145]:
from lightgbm import LGBMClassifier
import time

#lightgbm  leaf-wise : loss를 줄일 수 있음

start_time = time.time()

# 과적합제어하기 위해 reg_lambda, reg_alpha 적용
lgbm_clf = LGBMClassifier(n_estimators= 400, n_jobs=-1, reg_alpha = 0.9, reg_lambda = 0.2)
lgbm_clf.fit(tfidf_matrix_train, y_train)
print(f'fit time : {time.time() - start_time}')

fit time : 54.54272794723511


In [146]:
from sklearn.metrics import accuracy_score

tfidf_matrix_test = tfidf_vect.transform(x_test)
preds = lgbm_clf.predict(tfidf_matrix_test)
pred_proba = lgbm_clf.predict_proba(tfidf_matrix_test)

accuracy_score(y_test, preds)

0.7075874966604329

In [147]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

confusion = confusion_matrix(y_test, preds)
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='weighted')
recall = recall_score(y_test, preds, average='weighted')
f1 = f1_score(y_test, preds, average='weighted')

print("LGBM Classifier\n")
print(confusion)
print('정확도: {0:.4f}\n정밀도: {1:.4f}\n재현율: {2:.4f}\nf1: {3:.4f}'
      .format(accuracy,precision,recall,f1))

LGBM Classifier

[[1616  615  259]
 [ 181 5901 1157]
 [ 162 2004 3077]]
정확도: 0.7076
정밀도: 0.7119
재현율: 0.7076
f1: 0.7043


<p> 0 : 기쁨 행복</p>
<p> 1 : 분노 불안 당황 놀람</p>
<p> 2: 슬픔 상처 </p>

In [206]:
x_test = "ㅜㅜ"

In [207]:
lgbm_clf.predict(tfidf_vect.transform([x_test]))

array([1], dtype=int64)

## 다양한 분류 기법 비교

In [106]:
# 1. Decision tree
from sklearn.tree import DecisionTreeClassifier

# 예제 반복 시 마다 동일한 예측 결과 도출을 위해 random_state 설정
dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(tfidf_matrix_train, y_train)
pred = dt_clf.predict(tfidf_matrix_test)

In [107]:
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='weighted')
recall = recall_score(y_test, preds, average='weighted')
f1 = f1_score(y_test, preds, average='weighted')
print("DecisionTree Classifier\n")
print('정확도: {0:.4f}\n정밀도: {1:.4f}\n재현율: {2:.4f}\nf1: {3:.4f}'.format(accuracy,precision,recall,f1))

DecisionTree Classifier

정확도: 0.7008
정밀도: 0.7051
재현율: 0.7008
f1: 0.6972


In [108]:
# 2. RandomForest
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=156)
rf_clf.fit(tfidf_matrix_train, y_train)
pred = dt_clf.predict(tfidf_matrix_test)

In [109]:
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='weighted')
recall = recall_score(y_test, preds, average='weighted')
f1 = f1_score(y_test, preds, average='weighted')
print("RandomForest Classifier \n")
print('정확도: {0:.4f}\n정밀도: {1:.4f}\n재현율: {2:.4f}\nf1: {3:.4f}'.format(accuracy,precision,recall,f1))

RandomForest Classifier 

정확도: 0.7008
정밀도: 0.7051
재현율: 0.7008
f1: 0.6972


In [110]:
# 3. GBM 
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(tfidf_matrix_train, y_train)
gb_pred= gb_clf.predict(tfidf_matrix_test)

In [111]:
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='weighted')
recall = recall_score(y_test, preds, average='weighted')
f1 = f1_score(y_test, preds, average='weighted')
print("radientBoosting Classifier \n")
print('정확도: {0:.4f}\n정밀도: {1:.4f}\n재현율: {2:.4f}\nf1: {3:.4f}'.format(accuracy,precision,recall,f1))

radientBoosting Classifier 

정확도: 0.7008
정밀도: 0.7051
재현율: 0.7008
f1: 0.6972


<p> 0 : 기쁨 행복</p>
<p> 1 : 분노 불안 당황 놀람</p>
<p> 2: 슬픔 상처 </p>

In [112]:
data1.head()

,label,text
0,0,지금 난 기분이 너무 좋아.
1,0,나도 조카가 생겨! 너무 기뻐.
2,1,거래처와의 다음 계약이 무산될까봐 불안해.
3,2,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...
4,2,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.


# confusion matrix

[[1656  584  250] <br>
[ 177 5885 1177]<br>
[ 143 2133 2967]]

정확도: 0.7018<br>
정밀도: 0.7066<br>
재현율: 0.7018<br>
f1: 0.7018<br>